In [ ]:

%load_ext autoreload
%autoreload 2

In [34]:
import requests
import json # Import the json library

# Example: watch for the Intuit clickstream, wait 3s inactivity, max 20s overall
payload = {
    "watch_url": "https://eventbus.intuit.com/v2/segment/sbseg-qbo-clickstream/t",
    "silence_ms": 3000,
    "max_runtime_ms": 20000
}

print(f"Sending request to /listen_for_requests with payload: {payload}")
try:
    r = requests.post("http://127.0.0.1:8000/listen_for_requests", json=payload, timeout=25) # Add timeout
    print(f"\nStatus Code: {r.status_code}")

    if r.status_code == 200:
        response_data = r.json()
        print("\nRaw JSON Response:")
        print(json.dumps(response_data, indent=2)) # Pretty print the full response

        captured_requests = response_data.get('captured_requests', [])

        if not captured_requests:
            print("\nNo matching requests were captured.")
        else:
            print(f"\n--- Extracted Key Event Data ({len(captured_requests)} captured) ---")
            for i, req in enumerate(captured_requests):
                print(f"\nEvent {i+1}:")
                print(f"  Request URL: {req.get('request_url')}")
                print(f"  Origin Page URL: {req.get('page_url')}") # URL when event fired

                post_data_str = req.get('post_data')
                if post_data_str:
                    try:
                        # Parse the JSON string within post_data
                        post_data_json = json.loads(post_data_str)

                        # Extract key fields using .get() for safety
                        event_name = post_data_json.get('event', 'N/A')
                        event_type = post_data_json.get('type', 'N/A')

                        properties = post_data_json.get('properties', {}) # Get properties dict safely
                        ui_action = properties.get('ui_action', 'N/A')
                        object_detail = properties.get('object_detail', 'N/A')
                        screen = properties.get('screen', 'N/A')
                        ui_object_detail = properties.get('ui_object_detail', 'N/A')
                        ui_access_point = properties.get('ui_access_point', 'N/A')
                        event_sender = properties.get('event_sender_name', 'N/A')

                        # Print extracted fields
                        print(f"  Key Fields from post_data:")
                        print(f"    event:             {event_name}")
                        print(f"    type:              {event_type}")
                        print(f"    properties.ui_action:        {ui_action}")
                        print(f"    properties.object_detail:    {object_detail}") # Often indicates target
                        print(f"    properties.screen:           {screen}")        # Often indicates target screen
                        print(f"    properties.ui_object_detail: {ui_object_detail}")# e.g., leftnav
                        print(f"    properties.ui_access_point:  {ui_access_point}") # e.g., left_nav
                        print(f"    properties.event_sender_name:{event_sender}")   # Component sending event

                    except json.JSONDecodeError:
                        print("  Error: Could not decode post_data as JSON.")
                    except Exception as e:
                        print(f"  Error processing post_data: {e}")
                else:
                    print("  No post_data found for this request.")
            print("\n----------------------------------------------------")

    else:
        print(f"Error: Received status code {r.status_code}")
        try:
            print("Error details:", r.json())
        except requests.exceptions.JSONDecodeError:
            print("Error details:", r.text)

except requests.exceptions.RequestException as e:
    print(f"\nAn error occurred during the request: {e}")

Sending request to /listen_for_requests with payload: {'watch_url': 'https://eventbus.intuit.com/v2/segment/sbseg-qbo-clickstream/t', 'silence_ms': 3000, 'max_runtime_ms': 20000}

Status Code: 200

Raw JSON Response:
{
  "captured_requests": [
    {
      "method": "POST",
      "request_url": "https://eventbus.intuit.com/v2/segment/sbseg-qbo-clickstream/t",
      "page_url": "https://sandbox.qbo.intuit.com/app/banking",
      "request_headers": {
        "sec-ch-ua-platform": "\"Android\"",
        "referer": "https://sandbox.qbo.intuit.com/",
        "user-agent": "Mozilla/5.0 (Linux; Android 6.0; Nexus 5 Build/MRA58N) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/134.0.0.0 Mobile Safari/537.36",
        "sec-ch-ua": "\"Chromium\";v=\"134\", \"Not:A-Brand\";v=\"24\", \"Google Chrome\";v=\"134\"",
        "content-type": "text/plain",
        "sec-ch-ua-mobile": "?1"
      },
      "post_data": "{\"timestamp\":\"2025-04-06T04:04:13.009Z\",\"integrations\":{\"Segment.io\":true,\"Adobe 

In [23]:
r.json()#['captured_requests'][0]

{'captured_requests': [{'method': 'POST',
   'request_url': 'https://eventbus.intuit.com/v2/segment/sbseg-qbo-clickstream/t',
   'page_url': 'https://sandbox.qbo.intuit.com/app/banking',
   'post_data': '{"timestamp":"2025-04-06T02:35:25.702Z","integrations":{"Segment.io":true,"Adobe Analytics":{"marketingCloudVisitorId":"02123037359997510281562491858530907757","imsregion":9,"visitorId":"338B69C673615423-400011FF23D1CB88"},"Actions Amplitude":{"session_id":1743906270060}},"event":"navigation: engaged","type":"track","properties":{"dynamic":{"dynamic_id":"global_header_item_flyout"},"app_context":{"top_level_tracking_contexts":[{"name":"qbo-navigation-widgets-ui/globalHeader","version":"1.0.0"},{"name":"qbo-navigation-widgets-ui/leftNav","version":"1.0.0"},{"name":"ipd/after-navigation","version":"1.0.0"},{"name":"ipd/global-header","version":"1.0.0"},{"name":"theme-toggle/theme-toggle-enabler","version":"1.0.0"},{"name":"qbo-postload-entities/topicSubscriberWidget","version":"1.0.0"},{